In [50]:
import pandas as pd
import numpy as np

from typing import Callable

In [51]:
data = pd.read_csv("data/train.csv")

targets = data.columns[-3:]

In [52]:
def window_sliding(data: pd.DataFrame, features: dict[str, list[tuple]], in_place=False):
    if not in_place:
        data = data.copy()
    
    for feature, functions in features.items():
        for func, sizes in functions.items():
            for w_size in sizes:
                rolling = data[feature].rolling(window=w_size, min_periods=1)
                data[f"{feature}_{func.__name__}_{w_size}h"] = rolling.agg(func)
    
    if not in_place:
        return data

In [53]:
data_cp = data.copy()

In [54]:
sensors = data_cp.columns[4:9]

Зададим размеры окон в часах, основываясь на выделенных периодах. Воспользуемся функциями min, max, std. При необходимости можно изменить периоды или функции.

In [55]:
window_sliding(data_cp, dict(zip(list(targets) + list(sensors), [{np.max: [72, 48, 24], np.min: [72, 48, 24], np.std: [72, 48, 24], np.average: [72, 48, 24]}, 
                                                           {np.max: [72, 48, 24], np.min: [72, 48, 24], np.std: [72, 48, 24], np.average: [72, 48, 24]}, 
                                                           {np.max: [72, 48, 24], np.min: [72, 48, 24], np.std: [72, 48, 24], np.average: [72, 48, 24]},
                                                                {np.max: [24], np.min: [24], np.std: [24], np.average: [24]},
                                                                {np.max: [24], np.min: [24], np.std: [24], np.average: [24]},
                                                                {np.max: [24], np.min: [24], np.std: [24], np.average: [24]},
                                                                {np.max: [24], np.min: [24], np.std: [24], np.average: [24]},
                                                                {np.max: [24], np.min: [24], np.std: [24], np.average: [24]}])), in_place=True)

# TODO: horizont

In [56]:
data_cp.head(5)

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,...,sensor_3_std_24h,sensor_3_average_24h,sensor_4_amax_24h,sensor_4_amin_24h,sensor_4_std_24h,sensor_4_average_24h,sensor_5_amax_24h,sensor_5_amin_24h,sensor_5_std_24h,sensor_5_average_24h
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,...,NaN,1056.000000,1742.8,1742.8,NaN,1742.800000,1293.4,1293.4,NaN,1293.400000
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,...,100.055610,1126.750000,1742.8,1449.9,207.111576,1596.350000,1293.4,1010.9,199.757666,1152.150000
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,...,80.510020,1104.566667,1742.8,1449.9,146.569517,1592.933333,1293.4,1010.9,142.700397,1140.433333
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,...,65.741438,1104.150000,1742.8,1449.9,122.954988,1578.825000,1293.4,1010.9,131.694783,1171.125000
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,...,66.479809,1119.500000,1742.8,1415.5,129.125745,1546.160000,1293.4,1010.9,115.371868,1163.340000


Также можно продифферецнировать ряд

In [57]:
def diff(targets_with_n: dict):
    for t, n in targets_with_n.items():
        data_cp[f'{t}_diff'] = data_cp[f'{t}'].diff(n)

In [58]:
diff(dict(zip(targets, [1, 1, 1])))

Также сделаем отдельные параметры для дня, месяца, года. Также сделаем отдельный параметр для сезонов, как это было сделано в EDA

In [59]:
data_cp.date_time = pd.to_datetime(data.date_time)

In [60]:
interval_1 = data.date_time < data.date_time[3450]
interval_2 = (data.date_time > data.date_time[3450]) & (data.date_time < data.date_time[4200])
interval_3 = data.date_time > data.date_time[4200]

data_cp["season"] = np.nan
data_cp.loc[interval_1, 'season'], data_cp.loc[interval_2, 'season'], data_cp.loc[interval_3, 'season'] = 1, 2, 3

In [61]:
data_cp["date_time_hour"] = data_cp.date_time.dt.hour
data_cp["date_time_day"] = data_cp.date_time.dt.day
data_cp["date_time_month"] = data_cp.date_time.dt.month
data_cp["date_time_year"] = data_cp.date_time.dt.year

In [62]:
data_cp.iloc[:, 9:].head(5)

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,target_carbon_monoxide_amax_72h,target_carbon_monoxide_amax_48h,target_carbon_monoxide_amax_24h,target_carbon_monoxide_amin_72h,target_carbon_monoxide_amin_48h,target_carbon_monoxide_amin_24h,target_carbon_monoxide_std_72h,...,sensor_5_std_24h,sensor_5_average_24h,target_carbon_monoxide_diff,target_benzene_diff,target_nitrogen_oxides_diff,season,date_time_hour,date_time_day,date_time_month,date_time_year
0,2.5,12.0,167.7,2.5,2.5,2.5,2.5,2.5,2.5,NaN,...,NaN,1293.400000,NaN,NaN,NaN,1.0,18,10,3,2010
1,2.1,9.9,98.9,2.5,2.5,2.5,2.1,2.1,2.1,0.282843,...,199.757666,1152.150000,-0.4,-2.1,-68.8,1.0,19,10,3,2010
2,2.2,9.2,127.1,2.5,2.5,2.5,2.1,2.1,2.1,0.208167,...,142.700397,1140.433333,0.1,-0.7,28.2,1.0,20,10,3,2010
3,2.2,9.7,177.2,2.5,2.5,2.5,2.1,2.1,2.1,0.173205,...,131.694783,1171.125000,0.0,0.5,50.1,1.0,21,10,3,2010
4,1.5,6.4,121.8,2.5,2.5,2.5,1.5,1.5,1.5,0.367423,...,115.371868,1163.340000,-0.7,-3.3,-55.4,1.0,22,10,3,2010


Основываясь на графиках можно задать нелинейные параметры

In [63]:
data_cp['sens_sq'] = data['sensor_2']**2
data_cp['sens_hyp'] = 1 / data['sensor_3']

Также нам может понадобиться масштабирование признаков. Если этого не сделать, то в случае применения регуляризации модель будет работать хуже, так как у малых по модулю признаков, возможно, будут большие веса, а регуляризация "штрафует" за это. Также масштабирование повысит интерпретируемость, так как веса можно будет понимать как важность признака.

In [64]:
features = list(data_cp.columns[1:9]) + list(data_cp.columns[12:])

In [65]:
from sklearn.preprocessing import StandardScaler

X, Y = data_cp.dropna()[features], data_cp.dropna()[targets]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(X, Y)

In [66]:
%store features_scaled
%store Y
%store features
%store data_cp

Stored 'features_scaled' (ndarray)
Stored 'Y' (DataFrame)
Stored 'features' (list)
Stored 'data_cp' (DataFrame)
